# Statefulness

The first bots built within the course are stateless, meaning they have a 
consistent response to a specific kind of message. 

**Policies** - The relationship of a user input to an action.

Using memory (slot filling) & adding statefulness provides context to
the conversation. This can be useful if the user signals for assistance as 
they get stuck - you can be aware of where they were when you last
interacted with them.

<p style="color:red;">Warning: rasa_nlu dependency problems means this part
of the course is also non-reproducible.</p>

Below shows how to add statefulness to a bot, in the context of ordering a
meal from Mucky Bs...


In [ ]:
# Commonplace to use integers to signify states
# Define the initial state
START = 0

# Define the choice / choosing state
CHOOSE_MEAL = 1

# Define the state for when the order has been placed
PLACED = 2

# Set out the policy rules. Use the format dict(tuple:tuple)
# The key tuple should be the current state and the intent detected
# The value tuple should be the next state - the state to be progressed to
# and the message to respond with...
policy = {
    (START, "order"): (CHOOSE_MEAL, "ok, Crabby Patty or Heart Stopper Deluxe?"),
    (START, "none"): (START, "My apologies, I didn't understand that..."),
    (CHOOSE_MEAL, "specify_food"): (PLACED, "perfect, the meal is on its way!"),
    (CHOOSE_MEAL, "none"): (CHOOSE_MEAL, "Umm... You can choose from Crabby Patties or Heart Stoppers, sir..."),
}

# Here define a list of inputs form the user 
inputs = [
    "I'd like to shave my beard",
    "oh right, let me get a meal then",
    "I own a German Shephard (the dog)",
    "make it a Crabby Patty then"
]

# Call send_message() for each message
state = START
#for message in inputs:    
#    state = send_message(policy, state, message)

# send_message was abstracted away in the course. It has a rasa dependency.